In [ ]:
# ============================================================================
# CELLULE 1 : PRÉPARATION DONNÉES CONVERSATIONNELLES GYM-EXERCISE
# ============================================================================

print("🚀 CELLULE 1 : Données conversationnelles GYM-Exercise")
print("=" * 50)

# === INSTALLATION AUTO ===
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"📥 Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installé")

required_packages = ["datasets", "accelerate", "evaluate"]
for pkg in required_packages:
    install_if_missing(pkg)

# === IMPORTS ===
import torch
import pandas as pd
import numpy as np
from transformers import (
    MT5ForConditionalGeneration, 
    MT5Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
import json
from pathlib import Path
import logging
import re
from sklearn.model_selection import train_test_split
import random
import shutil

# Configuration
logging.basicConfig(level=logging.INFO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Device utilisé: {device}")

# === CHARGEMENT DATASET CONVERSATIONNEL ===
print(f"\n📥 Chargement du dataset GYM-Exercise conversationnel...")
gym_df = pd.read_parquet("hf://datasets/onurSakar/GYM-Exercise/data/train-00000-of-00001.parquet")
print(f"✅ Dataset chargé: {len(gym_df)} conversations")

# === PARSING DU FORMAT CONVERSATIONNEL ===
print(f"\n🔄 Parsing du format conversationnel...")

def parse_conversational_format(df):
    """Parse le format [INST]...[/INST] en questions-réponses"""
    parsed_data = []
    
    for idx, row in df.iterrows():
        text = str(row['text'])
        
        # Pattern pour extraire question et réponse
        # Format: <s>[INST] <<SYS>>...<<</SYS>> QUESTION [/INST]RÉPONSE</s>
        
        # Extraction de la question
        question_match = re.search(r'<<</SYS>>\s*(.*?)\s*\[/INST\]', text, re.DOTALL)
        if not question_match:
            continue
        
        question = question_match.group(1).strip()
        
        # Extraction de la réponse
        response_match = re.search(r'\[/INST\]\s*(.*?)(?:</s>|$)', text, re.DOTALL)
        if not response_match:
            continue
        
        response = response_match.group(1).strip()
        
        # Filtrage des questions/réponses de qualité
        if (len(question) > 10 and len(response) > 20 and 
            len(question) < 200 and len(response) < 1000):
            
            parsed_data.append({
                "input": question,
                "output": response
            })
    
    return parsed_data

# Parse du dataset
gym_parsed_data = parse_conversational_format(gym_df)
print(f"✅ {len(gym_parsed_data)} conversations parsées avec succès")

# Échantillon du parsing
print(f"\n🎯 Exemples de conversations parsées :")
for i in range(min(3, len(gym_parsed_data))):
    example = gym_parsed_data[i]
    print(f"\n--- Exemple {i+1} ---")
    print(f"❓ Question: {example['input'][:80]}...")
    print(f"🤖 Réponse: {example['output'][:100]}...")

# === AMÉLIORATION DES RÉPONSES POUR LE COACHING ===
print(f"\n🏋️ Transformation en style coaching professionnel...")

def enhance_gym_responses(parsed_data, max_examples=800):
    """Améliore les réponses avec un style coaching professionnel"""
    enhanced_data = []
    
    for example in parsed_data[:max_examples]:
        question = example['input']
        original_response = example['output']
        
        # Détection du type de question
        question_lower = question.lower()
        
        # Style coaching adapté selon le type
        if any(kw in question_lower for kw in ['build muscle', 'gain muscle', 'muscle building']):
            enhanced_response = f"""💪 **PRISE DE MUSCLE - Réponse Coach :**

{original_response}

✅ **Conseils additionnels :**
• Progression graduelle : +2.5kg/semaine
• Repos 48-72h entre séances même muscle
• Nutrition : 1.8-2.2g protéines/kg de poids"""
        
        elif any(kw in question_lower for kw in ['lose weight', 'weight loss', 'fat loss']):
            enhanced_response = f"""🔥 **PERTE DE POIDS - Réponse Coach :**

{original_response}

✅ **Stratégie complète :**
• Déficit calorique modéré (-300-500 cal/jour)
• Combinaison cardio + musculation
• Patience : 0.5-1kg/semaine = durable"""
        
        elif any(kw in question_lower for kw in ['exercise', 'workout', 'training']):
            enhanced_response = f"""🏋️ **ENTRAÎNEMENT - Réponse Coach :**

{original_response}

✅ **Points clés sécurité :**
• Échauffement 5-10min obligatoire
• Technique avant charge
• Écouter son corps = priorité"""
        
        elif any(kw in question_lower for kw in ['nutrition', 'diet', 'eating']):
            enhanced_response = f"""🍎 **NUTRITION - Réponse Coach :**

{original_response}

✅ **Règles d'or :**
• Régularité > perfection
• Hydratation : 35ml/kg de poids
• Légumes à chaque repas"""
        
        else:
            # Format général coaching
            enhanced_response = f"""💡 **CONSEIL COACH :**

{original_response}

✅ **Rappel important :** Consistance et patience sont vos meilleurs alliés !"""
        
        enhanced_data.append({
            "input": question,
            "output": enhanced_response
        })
    
    return enhanced_data

# Amélioration des réponses
gym_enhanced_data = enhance_gym_responses(gym_parsed_data)
print(f"✅ {len(gym_enhanced_data)} réponses améliorées en style coaching")

# === EXPERTISE COACHING ADDITIONNELLE ===
print(f"\n🧠 Ajout d'expertise coaching spécialisée...")

expert_coaching_data = [
    {
        "input": "Je débute complètement en sport, par où commencer ?",
        "output": """🚀 **PLAN DE DÉMARRAGE DÉBUTANT COMPLET :**

**📅 SEMAINES 1-2 : Création de l'habitude**
• 3 séances de 20 minutes (lundi, mercredi, vendredi)
• Marche rapide 15min + étirements 5min
• Objectif : RÉGULARITÉ, pas performance

**📅 SEMAINES 3-4 : Introduction exercices**
• Ajoutez 5 exercices au poids du corps
• Squats, pompes (genoux), planche, fentes, jumping jacks
• 2 séries de 8-12 répétitions chacun

**🎯 CLÉS DU SUCCÈS :**
✅ Commencez PETIT mais soyez CONSTANT
✅ Hydratation : 2L d'eau minimum/jour
✅ Sommeil : 7-8h pour récupération
✅ Célébrez chaque petite victoire !

💪 **MOTIVATION :** Vous venez de prendre la MEILLEURE décision de votre vie !"""
    },
    
    {
        "input": "Programme spécial femme débutante perte de poids",
        "output": """👩‍💪 **PROGRAMME FEMME PERTE DE POIDS OPTIMISÉ :**

**🔥 CARDIO ADAPTÉ (3x/semaine) :**
• Lundi : Marche rapide 30min + escaliers 10min
• Mercredi : Danse/Zumba 45min (fun + efficace)
• Vendredi : Vélo/elliptique 35min intensité modérée

**💪 RENFORCEMENT CIBLÉ (2x/semaine) :**
• Mardi : Haut du corps + core (focus tonification)
• Jeudi : Bas du corps + fessiers (sculpter)

**🍎 NUTRITION SIMPLE ET EFFICACE :**
• Petit-déj : Protéines + fruits + céréales complètes
• Déjeuner : Légumes + protéines maigres + féculents
• Dîner : Léger, privilégier légumes + protéines
• Collations : Fruits, yaourt grec, amandes

**📈 OBJECTIFS RÉALISTES :**
• Perte : 0.5-1kg/semaine maximum
• Mesures : Tour de taille, cuisses (+ important que balance)
• Photos : 1x/semaine même tenue, même pose

🌟 **VOUS ÊTES CAPABLE DE RÉUSSIR !**"""
    },
    
    {
        "input": "Je stagne depuis 2 mois, comment relancer ?",
        "output": """🔄 **STRATÉGIE ANTI-PLATEAU COMPLÈTE :**

**🎯 POURQUOI ÇA STAGNE :**
• Votre corps s'adapte = SIGNE DE PROGRÈS !
• Routine trop répétitive
• Possible sous-alimentation ou surentraînement

**💥 SOLUTIONS IMMÉDIATES :**

**1. SHOCK TRAINING (1 semaine) :**
• Changez COMPLÈTEMENT votre routine
• Nouveaux exercices, nouvelles amplitudes
• Intensité différente (force → endurance)

**2. SEMAINE DE DÉCHARGE :**
• Réduisez volume de 50%
• Gardez intensité mais moins de séries
• Plus de mobilité/étirements

**3. NUTRITION RESET :**
• Calculez vos calories réelles
• +200 cal si sous-alimentation
• Vérifiez protéines : 1.8-2g/kg

**4. PROGRESSION FORCÉE :**
• Techniques avancées : drop sets, rest-pause
• +2.5kg ou +2 reps OBLIGATOIRE chaque semaine

💡 **PATIENCE + PERSISTENCE = VICTOIRE GARANTIE !**"""
    },
    
    {
        "input": "Mal au dos après squats, que faire ?",
        "output": """⚠️ **PROTOCOLE URGENCE MAL DE DOS :**

**🚨 IMMÉDIATEMENT :**
• ARRÊT complet des squats
• Pas de charge sur le dos pendant 1 semaine
• Marche légère uniquement

**🔍 CAUSES PROBABLES :**
• Technique incorrecte (genoux qui rentrent)
• Manque mobilité chevilles/hanches
• Core trop faible
• Progression trop rapide

**🩺 PLAN DE RÉCUPÉRATION (2 semaines) :**

**Semaine 1 - Repos actif :**
• Étirements doux : psoas, ischio-jambiers
• Renforcement core : planche, dead bug
• Mobilité hanches : 10min/jour

**Semaine 2 - Reprise progressive :**
• Goblet squats (poids devant) légers
• Amplitude partielle puis complète
• Focus TECHNIQUE avant charge

**🔄 PRÉVENTION FUTURE :**
✅ Échauffement 10min minimum
✅ Mobilité quotidienne hanches/chevilles
✅ Renforcement core 3x/semaine
✅ Progression +5% max par semaine

**🚨 SI DOULEUR PERSISTE → Kinésithérapeute OBLIGATOIRE**

💡 **Rappel :** 2 semaines de repos > 6 mois de blessure !"""
    },
    
    {
        "input": "Plus motivé, je veux tout abandonner",
        "output": """🔥 **RECONQUÊTE MOTIVATION - STRATÉGIE PSYCHOLOGIQUE :**

**🧠 RÉALITÉ CHECK :**
• La motivation FLUCTUE chez TOUT LE MONDE
• Les champions ont des jours difficiles aussi
• Les HABITUDES battent la motivation

**💪 TECHNIQUES ANTI-ABANDON :**

**1. RÈGLE DES 2 MINUTES :**
• "Je fais juste 2 pompes"
• 90% du temps, vous continuerez !
• Maintenir l'habitude > performance

**2. SYSTÈME RÉCOMPENSES :**
• 1 semaine complète = film/massage
• 2 semaines = nouveau vêtement sport
• 1 mois = week-end détente
• 3 mois = objectif ÉNORME

**3. ACCOUNTABILITY POWER :**
• Partenaire d'entraînement
• Photos de progression
• Journal de victoires (même petites)

**4. RAPPEL PUISSANT :**
• Listez POURQUOI vous avez commencé
• Visualisez la version FORTE de vous-même
• Vous ne regrettez JAMAIS une séance faite

**🏆 MANTRA COACH :**
"Chaque jour, je choisis entre la version forte et faible de moi.
Aujourd'hui, je choisis la FORCE !"

💥 **VOUS AVEZ LE POUVOIR ! REPRENEZ-LE !**"""
    }
]

print(f"✅ {len(expert_coaching_data)} exemples d'expertise coaching ajoutés")

# === FUSION FINALE OPTIMISÉE ===
print(f"\n🔗 Fusion optimisée des datasets...")

# Vérification des données parsées
print(f"🔍 Vérification des données :")
print(f"   • Conversations GYM parsées : {len(gym_enhanced_data)}")
print(f"   • Expertise coaching : {len(expert_coaching_data)}")

if len(gym_enhanced_data) == 0:
    print(f"⚠️ Aucune conversation GYM parsée ! Utilisation expertise uniquement.")
    gym_sample = []
    final_dataset = expert_coaching_data
else:
    # Équilibrage intelligent : 70% GYM parsé + 30% expertise
    total_expert = len(expert_coaching_data)
    target_gym_count = int(total_expert * 2.33)  # Pour ratio 70/30
    
    if len(gym_enhanced_data) > target_gym_count:
        gym_sample = random.sample(gym_enhanced_data, target_gym_count)
    else:
        gym_sample = gym_enhanced_data
    
    # Fusion et mélange
    final_dataset = expert_coaching_data + gym_sample

random.shuffle(final_dataset)

print(f"📊 Dataset final optimisé :")
print(f"   • Expertise coaching spécialisée : {len(expert_coaching_data)} exemples")
print(f"   • GYM conversationnel amélioré : {len(gym_sample)} exemples")
print(f"   • TOTAL : {len(final_dataset)} exemples")

# Calcul du ratio sécurisé
if len(gym_sample) > 0:
    ratio_expertise = (len(expert_coaching_data) / len(gym_sample)) * 100
    print(f"   • Ratio expertise/gym : {len(expert_coaching_data)}/{len(gym_sample)} = {ratio_expertise:.0f}%/70%")
else:
    print(f"   • Ratio : 100% expertise (pas de données GYM parsées)")

# Diagnostic si problème de parsing
if len(gym_enhanced_data) == 0:
    print(f"\n🔍 DIAGNOSTIC PARSING :")
    print(f"   • Dataset original : {len(gym_df)} lignes")
    print(f"   • Parsing réussi : {len(gym_parsed_data)} conversations")
    print(f"   • Amélioration réussie : {len(gym_enhanced_data)} conversations")
    
    if len(gym_parsed_data) == 0:
        print(f"   ❌ Problème : Aucune conversation parsée du format [INST]...[/INST]")
        print(f"   💡 Solution : Vérification du format des données")
        
        # Affichage d'un échantillon pour debug
        print(f"\n📄 Échantillon des données brutes :")
        for i in range(min(2, len(gym_df))):
            sample_text = str(gym_df.iloc[i]['text'])[:200]
            print(f"   Ligne {i+1}: {sample_text}...")
    
    print(f"   💪 Pas de problème ! L'expertise coaching suffit pour l'entraînement.")

# === VALIDATION QUALITÉ (suite du code inchangée) ===
print(f"\n🎯 Validation de la qualité :")

# Analyse des mots-clés coaching
coaching_keywords = ['coach', 'conseil', 'progression', 'sécurité', 'technique', 'motivation']
keyword_coverage = 0

sample_size = min(50, len(final_dataset))
for example in final_dataset[:sample_size]:
    response_lower = example['output'].lower()
    if any(kw in response_lower for kw in coaching_keywords):
        keyword_coverage += 1

if sample_size > 0:
    coverage_percentage = (keyword_coverage / sample_size) * 100
    print(f"   • Couverture mots-clés coaching : {coverage_percentage:.0f}%")
    print(f"   • Longueur moyenne réponses : {np.mean([len(ex['output']) for ex in final_dataset]):.0f} caractères")
    print(f"   • Qualité : {'✅ Excellente' if coverage_percentage > 70 else '⚠️ À améliorer'}")
else:
    print(f"   ❌ Aucune donnée à valider")

# Aperçu final
if len(final_dataset) > 0:
    print(f"\n🎯 Aperçu du dataset final :")
    for i in range(min(2, len(final_dataset))):
        example = final_dataset[i]
        print(f"\n--- Exemple {i+1} ---")
        print(f"❓ Question: {example['input'][:70]}...")
        print(f"🤖 Réponse: {example['output'][:120]}...")

    print(f"\n✅ CELLULE 1 TERMINÉE - Dataset conversationnel prêt pour l'entraînement !")
    print(f"🎯 {len(final_dataset)} exemples de coaching de haute qualité préparés")
else:
    print(f"\n❌ PROBLÈME : Dataset final vide")
    print(f"💡 Vérifiez le parsing ou utilisez uniquement l'expertise coaching")

In [ ]:
# ============================================================================
# CELLULE 2 : CONFIGURATION ET ENTRAÎNEMENT MT5
# ============================================================================

print("🤖 CELLULE 2 : Configuration et entraînement MT5")
print("=" * 50)

# === PRÉPARATION POUR MT5 ===
print("\n🔄 Préparation des données pour MT5...")

def create_mt5_prompts(dataset):
    """Crée des prompts optimisés pour MT5"""
    mt5_dataset = []
    
    for example in dataset:
        # Prompt structuré pour MT5 avec préfixe coaching
        mt5_prompt = f"coach fitness expert: {example['input']}"
        
        mt5_dataset.append({
            "input": mt5_prompt,
            "output": example['output']
        })
    
    return mt5_dataset

# Conversion des données
mt5_dataset = create_mt5_prompts(final_dataset)

# Division train/validation stratifiée
train_data, val_data = train_test_split(mt5_dataset, test_size=0.15, random_state=42)

print(f"📊 Données MT5 préparées :")
print(f"   • Entraînement : {len(train_data)} exemples")
print(f"   • Validation : {len(val_data)} exemples")

# Vérification de la qualité des données
if len(train_data) < 10:
    print(f"⚠️ Dataset petit ({len(train_data)} exemples d'entraînement)")
    print(f"💡 L'entraînement sera rapide mais moins robuste")
else:
    print(f"✅ Dataset suffisant pour un bon entraînement")

# === CHARGEMENT MODÈLE MT5 ===
print(f"\n🤖 Chargement du modèle MT5...")

model_name = "google/mt5-base"
try:
    tokenizer = MT5Tokenizer.from_pretrained(model_name)
    model = MT5ForConditionalGeneration.from_pretrained(model_name)
    print(f"✅ Modèle {model_name} chargé avec succès")
except Exception as e:
    print(f"❌ Erreur chargement modèle : {e}")
    print(f"💡 Vérifiez votre connexion internet")
    raise

# Paramètres optimisés pour coaching
max_input_length = 512
max_target_length = 400  # Plus long pour réponses détaillées

print(f"⚙️ Configuration :")
print(f"   • Input max : {max_input_length} tokens")
print(f"   • Output max : {max_target_length} tokens")
print(f"   • Device : {device}")

# Transfert du modèle sur le device approprié
model.to(device)
print(f"✅ Modèle transféré sur {device}")

# === PRÉPROCESSING POUR ENTRAÎNEMENT ===
print(f"\n🔄 Configuration du preprocessing...")

def preprocess_mt5_coaching(examples):
    """Préprocessing optimisé pour coaching MT5"""
    
    inputs = examples["input"]
    targets = examples["output"]
    
    # Tokenisation inputs
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding=True
    )
    
    # Tokenisation targets
    labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True,
        padding=True
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Conversion en datasets Hugging Face
print("🔄 Tokenisation des données...")
try:
    train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
    val_dataset = Dataset.from_pandas(pd.DataFrame(val_data))
    
    tokenized_train = train_dataset.map(preprocess_mt5_coaching, batched=True)
    tokenized_val = val_dataset.map(preprocess_mt5_coaching, batched=True)
    print("✅ Tokenisation terminée")
except Exception as e:
    print(f"❌ Erreur tokenisation : {e}")
    raise

# === CONFIGURATION ENTRAÎNEMENT ===
print("\n⚙️ Configuration de l'entraînement...")

# Arguments d'entraînement optimisés pour coaching
# Adaptation selon la taille du dataset
if len(train_data) < 50:
    # Dataset très petit
    num_epochs = 8
    batch_size = 1
    eval_steps = 10
    save_steps = 20
    warmup_steps = 5
    print("📊 Configuration adaptée : Dataset très petit")
elif len(train_data) < 200:
    # Dataset petit
    num_epochs = 6
    batch_size = 2
    eval_steps = 25
    save_steps = 50
    warmup_steps = 20
    print("📊 Configuration adaptée : Dataset petit")
else:
    # Dataset standard
    num_epochs = 4
    batch_size = 2
    eval_steps = 100
    save_steps = 200
    warmup_steps = 100
    print("📊 Configuration adaptée : Dataset standard")

training_args = TrainingArguments(
    output_dir="./mt5-expert-fitness-coach",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=0.01,
    learning_rate=3e-5,  # Learning rate optimal pour fine-tuning
    logging_dir="./logs",
    logging_steps=max(10, len(train_data) // 10),  # Adaptatif
    evaluation_strategy="steps",
    eval_steps=eval_steps,
    save_steps=save_steps,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    gradient_accumulation_steps=max(1, 4 // batch_size),  # Simule batch_size plus grand
    fp16=torch.cuda.is_available(),  # Optimisation GPU si disponible
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to=None,  # Désactive wandb
    disable_tqdm=False,  # Affichage des barres de progression
)

print(f"✅ Configuration terminée :")
print(f"   • Époques : {training_args.num_train_epochs}")
print(f"   • Batch size : {training_args.per_device_train_batch_size}")
print(f"   • Learning rate : {training_args.learning_rate}")
print(f"   • Gradient accumulation : {training_args.gradient_accumulation_steps}")
print(f"   • Évaluation toutes les {training_args.eval_steps} étapes")

# Data collator
try:
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True
    )
    print("✅ Data collator configuré")
except Exception as e:
    print(f"❌ Erreur data collator : {e}")
    raise

# === INITIALISATION TRAINER ===
print(f"\n🏋️ Initialisation du Trainer...")

try:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        data_collator=data_collator,
        tokenizer=tokenizer
    )
    print("✅ Trainer configuré et prêt")
except Exception as e:
    print(f"❌ Erreur configuration Trainer : {e}")
    raise

# Estimation du temps d'entraînement
steps_per_epoch = len(tokenized_train) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
total_steps = steps_per_epoch * training_args.num_train_epochs
estimated_time_gpu = total_steps * 2 / 60  # ~2 secondes par step sur GPU
estimated_time_cpu = total_steps * 10 / 60  # ~10 secondes par step sur CPU

print(f"\n📊 Estimation entraînement :")
print(f"   • Steps par époque : {steps_per_epoch}")
print(f"   • Steps total : {total_steps}")
if torch.cuda.is_available():
    print(f"   • Temps estimé (GPU) : {estimated_time_gpu:.1f} minutes")
else:
    print(f"   • Temps estimé (CPU) : {estimated_time_cpu:.1f} minutes")

# === ENTRAÎNEMENT ===
print(f"\n🚀 DÉBUT DE L'ENTRAÎNEMENT DU COACH EXPERT")
print("=" * 50)
print("⏱️ L'entraînement va commencer...")
print("💡 Surveillez eval_loss : doit diminuer progressivement")
print("🔄 Vous pouvez interrompre avec Ctrl+C si nécessaire")

try:
    # Lancement de l'entraînement
    trainer.train()
    print("\n🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
    
except KeyboardInterrupt:
    print("\n⏹️ Entraînement interrompu par l'utilisateur")
    print("💾 Sauvegarde du modèle en cours d'entraînement...")
    
except Exception as e:
    print(f"\n❌ Erreur pendant l'entraînement : {e}")
    print("💾 Tentative de sauvegarde...")

# === SAUVEGARDE ===
print(f"\n💾 Sauvegarde du modèle expert...")

model_save_path = "./mt5-expert-fitness-coach"

try:
    # Création du dossier si nécessaire
    Path(model_save_path).mkdir(exist_ok=True)
    
    # Sauvegarde du modèle et tokenizer
    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    
    # Configuration de l'entraînement
    training_config = {
        "model_name": "mt5-expert-fitness-coach",
        "base_model": "google/mt5-base",
        "training_examples": len(train_data),
        "validation_examples": len(val_data),
        "total_examples": len(final_dataset),
        "max_input_length": max_input_length,
        "max_target_length": max_target_length,
        "epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "learning_rate": training_args.learning_rate,
        "training_date": pd.Timestamp.now().isoformat(),
        "device_used": str(device),
        "performance_notes": "Coach expert avec dataset conversationnel + expertise professionnelle"
    }
    
    with open(f"{model_save_path}/training_config.json", "w", encoding='utf-8') as f:
        json.dump(training_config, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Modèle expert sauvegardé dans : {model_save_path}")
    print("✅ Configuration sauvegardée")
    
    # Affichage du chemin absolu pour référence
    absolute_path = Path(model_save_path).resolve()
    print(f"📁 Chemin absolu : {absolute_path}")
    
except Exception as e:
    print(f"❌ Erreur sauvegarde : {e}")

# === VÉRIFICATION POST-ENTRAÎNEMENT ===
print(f"\n🔍 Vérification post-entraînement...")

# Vérification des fichiers sauvegardés
expected_files = ["config.json", "pytorch_model.bin", "tokenizer.json", "training_config.json"]
missing_files = []

for file in expected_files:
    file_path = Path(model_save_path) / file
    if file_path.exists():
        file_size = file_path.stat().st_size / (1024*1024)  # MB
        print(f"   ✅ {file} ({file_size:.1f} MB)")
    else:
        missing_files.append(file)
        print(f"   ❌ {file} manquant")

if missing_files:
    print(f"⚠️ Fichiers manquants : {missing_files}")
    print(f"💡 L'entraînement a peut-être été interrompu")
else:
    print(f"✅ Tous les fichiers essentiels présents")

print(f"\n✅ CELLULE 2 TERMINÉE - Modèle expert entraîné et sauvegardé !")
print(f"🎯 Prêt pour les tests et l'export Django (Cellule 3)")

message.txt